In [1]:
import json
import re
import requests
from tqdm import tqdm
import time
from concurrent.futures import ThreadPoolExecutor, as_completed
MAX_THREADS = 10 
WIKIDATA_SPARQL_ENDPOINT = "https://query.wikidata.org/sparql"
WIKIDATA_API_ENDPOINT = "https://www.wikidata.org/w/api.php"

def query_wikidata(sparql_query):
    """Truy vấn Wikidata và trả về kết quả phù hợp với từng loại truy vấn"""
    headers = {"User-Agent": "Mozilla/5.0", "Accept": "application/sparql-results+json"}
    response = requests.get(WIKIDATA_SPARQL_ENDPOINT, params={"query": sparql_query, "format": "json"}, headers=headers)

    if response.status_code == 200:
        data = response.json()

        # Xử lý truy vấn ASK (boolean)
        if "boolean" in data:
            return data["boolean"]

        # Xử lý truy vấn SELECT (trả về bindings)
        results = data.get("results", {}).get("bindings", [])
        answers = []

        for result in results:
            for var in result:  # Duyệt qua tất cả các biến trả về
                value = result[var]["value"]
                answers.append(value)  # Chấp nhận tất cả giá trị, không chỉ thực thể Wikidata

        return answers  # Trả về toàn bộ danh sách kết quả

    return None  # Trả về None nếu lỗi xảy ra
 # Trả về danh sách rỗng nếu có lỗi


def get_wikidata_label(entity_id):
    """ Lấy nhãn (label) tiếng Việt hoặc tiếng Anh của một thực thể/quan hệ """
    params = {
        "action": "wbgetentities",
        "ids": entity_id,
        "languages": "vi|en",
        "format": "json",
    }
    response = requests.get(WIKIDATA_API_ENDPOINT, params=params)
    if response.status_code == 200:
        data = response.json()
        labels = data.get("entities", {}).get(entity_id, {}).get("labels", {})
        return labels.get("vi", {}).get("value") or labels.get("en", {}).get("value") or entity_id
    return entity_id

def extract_entities_relations(sparql_query):
    """ Trích xuất các thực thể (QID) và quan hệ (PID) từ SPARQL """
    entities = set(re.findall(r"wd:(Q\d+)", sparql_query))
    relations = set(re.findall(r"wdt:(P\d+)|p:(P\d+)|ps:(P\d+)|pq:(P\d+)", sparql_query))
    relations = {r for t in relations for r in t if r}  # Xử lý tuple match
    return list(entities), list(relations)

def convert_s_expression_to_nor_s_expr(s_expression, entity_map, relation_map):
    """ Chuyển s_expression thành nor_s_expr bằng cách thay thế mã Q/P bằng nhãn """
    def replace_match(match):
        code = match.group(0)
        return f"[ {entity_map.get(code, relation_map.get(code, code))} ]"

    return re.sub(r"Q\d+|P\d+", replace_match, s_expression)

def preprocess_question(question):
    """ Loại bỏ dấu `{}` trong câu hỏi """
    return re.sub(r"[{}]", "", question)



In [ ]:
def process_test_file(input_file, output_file):
    with open(input_file, 'r', encoding='utf-8') as f:
        data = json.load(f)

    processed_data = []
    question_id = 1  # Biến đếm tự động

    print(f"🔄 Bắt đầu xử lý {len(data)} điểm dữ liệu với {MAX_THREADS} luồng...\n")
    start_time = time.time()  # Bắt đầu đo thời gian

    with ThreadPoolExecutor(max_workers=MAX_THREADS) as executor:
        future_to_entry = {executor.submit(query_wikidata, entry.get("sparql_wikidata", "")): entry for entry in data}
        
        for future in tqdm(as_completed(future_to_entry), total=len(data), desc="⏳ Đang xử lý", unit="câu"):
            entry = future_to_entry[future]
            try:
                answers = future.result()
                if not answers:
                    continue  # Bỏ qua điểm dữ liệu không có câu trả lời

                # Xử lý tiếp nếu có câu trả lời
                sparql_query = entry.get("sparql_wikidata", "")
                question = preprocess_question(entry.get("question", ""))
                s_expression = entry.get("s_expression", "")
                entities, relations = extract_entities_relations(sparql_query)

                # Truy vấn song song để lấy nhãn thực thể và quan hệ
                entity_map = {}
                relation_map = {}

                entity_futures = {executor.submit(get_wikidata_label, qid): qid for qid in entities}
                relation_futures = {executor.submit(get_wikidata_label, pid): pid for pid in relations}

                for future in as_completed(entity_futures):
                    entity_map[entity_futures[future]] = future.result()

                for future in as_completed(relation_futures):
                    relation_map[relation_futures[future]] = future.result()

                # Chuyển s_expression thành nor_s_expr
                nor_s_expr = convert_s_expression_to_nor_s_expr(s_expression, entity_map, relation_map) if s_expression else ""

                processed_data.append({
                    "question_id": question_id,
                    "question": question,
                    "sparql": sparql_query,
                    "answer": answers,
                    "s_expr": s_expression,
                    "nor_s_expr": nor_s_expr,
                    "gold_entity_map": entity_map,
                    "gold_relation_map": relation_map
                })
                question_id += 1  # Tăng số thứ tự

            except Exception as e:
                print(f"❌ Lỗi khi xử lý câu hỏi {question_id}: {e}")

    end_time = time.time()  # Kết thúc đo thời gian
    total_time = end_time - start_time

    with open(output_file, 'w', encoding='utf-8') as f:
        json.dump(processed_data, f, indent=4, ensure_ascii=False)

    print(f"\n✅ Xử lý hoàn tất! Kết quả được lưu vào {output_file}")
    print(f"⏳ Tổng thời gian xử lý: {total_time:.2f} giây ({total_time/60:.2f} phút)\n")

# Chạy chương trình
input_file = "s_expression/test_with_s_expression.json"
output_file = "filter/test_filtered.json"
process_test_file(input_file, output_file)


🔄 Bắt đầu xử lý 5590 điểm dữ liệu với 10 luồng...



⏳ Đang xử lý:  29%|██▉       | 1617/5590 [26:16<59:09,  1.12câu/s]  

❌ Lỗi khi xử lý câu hỏi 1274: expected string or bytes-like object


⏳ Đang xử lý:  38%|███▊      | 2112/5590 [32:49<1:00:45,  1.05s/câu]

❌ Lỗi khi xử lý câu hỏi 1656: expected string or bytes-like object


...

❌ Lỗi khi xử lý câu hỏi 3731: expected string or bytes-like object


⏳ Đang xử lý:  90%|████████▉ | 5012/5590 [1:11:23<08:34,  1.12câu/s]

❌ Lỗi khi xử lý câu hỏi 3983: expected string or bytes-like object


⏳ Đang xử lý: 100%|██████████| 5590/5590 [1:19:04<00:00,  1.18câu/s]



✅ Xử lý hoàn tất! Kết quả được lưu vào test_filtered.json
⏳ Tổng thời gian xử lý: 4744.77 giây (79.08 phút)



In [ ]:
input_file2 = "s_expression/train_with_s_expression.json"
output_file2 = "filter/train_filtered.json"
process_test_file(input_file2, output_file2)

🔄 Bắt đầu xử lý 22462 điểm dữ liệu với 10 luồng...



⏳ Đang xử lý:   2%|▏         | 501/22462 [21:54<5:33:06,  1.10câu/s]  

❌ Lỗi khi xử lý câu hỏi 385: expected string or bytes-like object


⏳ Đang xử lý:   3%|▎         | 576/22462 [22:51<4:10:25,  1.46câu/s]

❌ Lỗi khi xử lý câu hỏi 442: expected string or bytes-like object


...

❌ Lỗi khi xử lý câu hỏi 16923: expected string or bytes-like object


⏳ Đang xử lý:  98%|█████████▊| 22030/22462 [5:08:51<05:10,  1.39câu/s]

❌ Lỗi khi xử lý câu hỏi 17302: expected string or bytes-like object


⏳ Đang xử lý: 100%|██████████| 22462/22462 [5:14:06<00:00,  1.19câu/s]


❌ Lỗi khi xử lý câu hỏi 17614: Expecting ':' delimiter: line 15270484 column 17 (char 360513536)

✅ Xử lý hoàn tất! Kết quả được lưu vào train_filtered.json
⏳ Tổng thời gian xử lý: 18846.94 giây (314.12 phút)

